In [12]:
%reload_ext autoreload
%autoreload 2

import polars as pl
import panel as pn
import plotly.graph_objects as go
import plotly.subplots as subp

In [13]:
pems_holidays = (
    pl.read_excel("../data/PeMS/PeMS Holidays.xlsx")
)

pems_holidays.head(2)

date,holiday,weekday
date,str,str
2025-01-01,"""New Year's Day""","""Wednesday"""
2025-01-20,"""Martin Luther King, Jr. Day""","""Monday"""


In [14]:
from polars import map_batches


daily_vmt = (
    # Read all excel files in export folder.
    pl.read_excel("../data/PeMS/Aggregate Daily VMT Export/*", sheet_name="Report Data")
    .rename(
        {
            "Day": "date",
            "VMT (Veh-Miles)": "vmt",
            "# Lane Points": "observations",
            "% Observed": "pct_observed"
        }
    )
    # Drop any duplicate dates.
    .unique(subset="date", keep="last")
    .sort("date")
    .with_columns(
        weekday=(
            pl.col("date").dt.weekday()
            .map_elements(
                lambda x: {
                    1: "Monday",
                    2: "Tuesday",
                    3: "Wednesday",
                    4: "Thursday",
                    5: "Friday",
                    6: "Saturday",
                    7: "Sunday",
                }.get(x),
                return_dtype=pl.String,
            )
            .cast(
                pl.Enum(
                    categories=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
                )
            )
        )
    )
    .with_columns(
        is_weekend=pl.col("weekday").is_in(["Saturday", "Sunday"]),
        is_holiday=pl.col("date").is_in(pems_holidays["date"].to_list()),
    )
)
daily_vmt.tail(2)

date,vmt,observations,pct_observed,weekday,is_weekend,is_holiday
date,f64,i64,f64,enum,bool,bool
2025-03-30,3.3752e7,870336,55.7,"""Sunday""",true,false
2025-03-31,3.7520e7,870336,55.9,"""Monday""",false,true


In [15]:
annual_weekday_average_vmt = (
    daily_vmt
    .filter((~pl.col("is_weekend")) & (~pl.col("is_holiday")))
    .filter(pl.col("pct_observed") > 45.0)
    .sort(by="date")
    .group_by_dynamic(pl.col("date"), every="1y")
    .agg(
        mean=pl.col("vmt").mean(),
        std=pl.col("vmt").std(),
    )
    .with_columns(
        cv=pl.col("std")/pl.col("mean")
    )
    .sort("date")
    # Filter out 2025 since it's not a complete year
    .filter(pl.col("date").dt.year() != 2025)
)
annual_weekday_average_vmt

date,mean,std,cv
date,f64,f64,f64
2003-01-01,2.9076e7,1.4970e6,0.051486
2004-01-01,3.0646e7,1.7783e6,0.058027
2005-01-01,3.3463e7,1.6408e6,0.049033
2006-01-01,3.5203e7,1.3011e6,0.03696
2007-01-01,3.7101e7,2.4124e6,0.065021
…,…,…,…
2020-01-01,3.4109e7,5.3566e6,0.157042
2021-01-01,3.8312e7,2.8744e6,0.075026
2022-01-01,3.9532e7,1.6501e6,0.041741


In [16]:
data_table = (
    annual_weekday_average_vmt
    .select(
        pl.col("date").dt.year().cast(pl.String).alias("Year"),
        (
            pl.col("mean")
            .mul(1/1_000_000)
            .round(1)
            .cast(pl.String)
            .alias("Mean Weekday VMT")
            + "M"
        ),
        (
            pl.col("std")
            .mul(1/1_000_000)
            .round(1)
            .cast(pl.String)
            .alias("Standard Deviation Weekday VMT")
            + "M"
        ),
        (
            pl.col("cv")
            .mul(100)
            .round(1)
            .cast(pl.String)
            .alias("Coefficient of Variation")
            + "%"
        ),
        
        
    )
    .transpose(
        include_header=True,
        header_name="Statistic",
        column_names=[str(year) for year in range(2003, 2025)]
    )
    [1:]
)
data_table

Statistic,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""Mean Weekday VMT""","""29.1M""","""30.6M""","""33.5M""","""35.2M""","""37.1M""","""36.0M""","""33.8M""","""34.0M""","""33.5M""","""35.4M""","""37.0M""","""38.1M""","""40.4M""","""40.5M""","""42.2M""","""41.9M""","""41.9M""","""34.1M""","""38.3M""","""39.5M""","""40.3M""","""40.1M"""
"""Standard Deviation Weekday VMT""","""1.5M""","""1.8M""","""1.6M""","""1.3M""","""2.4M""","""2.1M""","""1.4M""","""1.7M""","""1.3M""","""2.4M""","""1.6M""","""1.8M""","""1.7M""","""1.6M""","""1.7M""","""1.9M""","""1.9M""","""5.4M""","""2.9M""","""1.7M""","""1.7M""","""1.8M"""
"""Coefficient of Variation""","""5.1%""","""5.8%""","""4.9%""","""3.7%""","""6.5%""","""5.9%""","""4.2%""","""4.9%""","""3.8%""","""6.8%""","""4.4%""","""4.6%""","""4.2%""","""3.9%""","""4.1%""","""4.5%""","""4.5%""","""15.7%""","""7.5%""","""4.2%""","""4.1%""","""4.5%"""


In [17]:
rolling_vmt = (
    daily_vmt
    .filter((~pl.col("is_weekend") & ~pl.col("is_holiday")))
    .filter(pl.col("pct_observed") > 45.0)
    .rolling("date", period="100d")
    .agg(
        (pl.col("vmt").mean() - pl.col("vmt").std()).alias("lower"),
        (pl.col("vmt").mean() + pl.col("vmt").std()).alias("upper"),
    )
)


In [18]:
figure = subp.make_subplots(
    rows=1, 
    cols=1,
    specs=[
        [{"type": "scatter"}],
    ],
)
figure.update_layout(
    {
        "hoversubplots": "axis",
        "title": "Daily Freeway VMT in San Diego Region",
        "hovermode": "x",
        "height": 1000,
        "width": 1_800,
        "grid": dict(rows=2, columns=1),
    }
)

figure.add_trace(
    go.Scatter(
        x=daily_vmt.filter((~pl.col("is_weekend") & ~pl.col("is_holiday")))["date"],
        y=daily_vmt.filter((~pl.col("is_weekend") & ~pl.col("is_holiday")))["vmt"],
        mode="markers",
        name="Weekday VMT",
        marker=dict(color='#717074', size=5),
        hovertemplate="Date: %{x|%Y-%m-%d}<br>Day: %{x|%A}<br>VMT: %{y:.3s}"
    )
)

figure.add_trace(
    go.Scatter(
        x=rolling_vmt["date"],
        y=rolling_vmt["upper"],
        mode="lines",
        name="Upper Standard Deviation",
        line=dict(color="#283C75", width=2),
        hovertemplate="Date: %{x|%Y-%m-%d}<br>Day: %{x|%A}<br>VMT: %{y:.3s}",
    )
)

figure.add_trace(
    go.Scatter(
        x=rolling_vmt["date"],
        y=rolling_vmt["lower"],
        mode="lines",
        name="Lower Standard Deviation",
        line=dict(color="#AF2B7C", width=2),
        hovertemplate="Date: %{x|%Y-%m-%d}<br>Day: %{x|%A}<br>VMT: %{y:.3s}",
    )
)

figure.add_trace(
    go.Scatter(
        x=annual_weekday_average_vmt["date"],
        y=annual_weekday_average_vmt["mean"],
        mode="markers",
        name="Annual Weekday Average VMT",
        marker_symbol="square",
        marker=dict(color='#8330C1', size=16, line={"color": "white", "width": 1}),
        hovertemplate="Year: %{x|%Y}<br>Day: %{x|%A}<br>VMT: %{y:.3s}",
        xperiod="M12",
        xperiodalignment="middle"
    )
)


""" figure.add_trace(
    go.Table(
        header={
            "values": ["Statistic"] + [str(year) for year in range(2003, 2025)],
            "font": {"size": 16},
            "align": "left",
        },
        cells={
            "values": [data_table[c].to_list() for c in data_table.columns],
            "align": "left",
        },
    ),
    row=2, col=1,
)
"""

figure.update_xaxes(
    dtick="M12",
    tickformat="%Y",
    tickfont={"size": 18},
    range=["2002-06-01", "2025-12-01"],
)
figure.update_yaxes(
    tickfont={"size": 18},
)


figure.update_layout(
    hovermode='x unified',
    legend={
        "yanchor": "top",
        "y": 1.00,
        "xanchor": "left",
        "x": 0.00,
        "orientation": "h",
        "font": {"size": 18}
    },
    title={"font": {"size": 36}},
    font_family="Montserrat",
    title_font_family="Montserrat SemiBold",
    title_font_color="#283C75",
)



In [ ]:
import panel as pn

pn.extension("plotly")

printout = pn.Row(
    pn.pane.plotly.Plotly(figure)
)

printout.save("../printout/printout.html")

FileNotFoundError: [Errno 2] No such file or directory: 'printout/printout.html'

In [ ]:
import plotly

plotly.io.write_image(figure, 'printout/printout.pdf', format='pdf')

In [ ]:
data_table.write_excel("printout/table.xlsx")